# INFO-4604 Final Project 

* Created by Garrett Glissmann on November 17, 2017

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dfMovies = pd.read_csv('tmdb_5000_movies.csv', sep=',', header=0)
# dfMovies = pd.read_csv('tmdb_5000_movies_out.csv', sep=',', header=0)
# dfMovies = pd.read_csv('tmdb_5000_movies_out2.csv', sep=',', header=0)
# dfMovies = pd.read_csv('tmdb_5000_movies_out3.csv', sep=',', header=0, encoding='latin1')
# dfMovies = pd.read_csv('tmdb_5000_movies_out4.csv', sep=',', header=0, encoding='latin1')
print("Number of columns = {}".format(len(dfMovies.columns)))
dfMovies.head()

Number of columns = 20


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [8]:
# Genres, production countries, spoken_languages

def json_to_cols (i, row, col_key, json_key):
    if (row[col_key]):
        gen = pd.read_json(row[col_key])
        if (json_key in gen.columns):
            for v in gen[json_key]:
                col = col_key + "_" + v.replace(' ', '_').lower()
                dfMovies.loc[i, col] = 1

for i, row in dfMovies.iterrows():
    # JSON fields
    json_to_cols(i, row, "genres", "name")
    json_to_cols(i, row, "production_countries", "iso_3166_1")
    json_to_cols(i, row, "spoken_languages", "iso_639_1")
    # Categorical value
    lang = row['original_language']
    if (lang):
        dfMovies.loc[i, 'original_language_' + lang] = 1

do_columns = ["genres", "production_countries", "spoken_languages", "original_language"]
for col_name in do_columns:
    col_name2 = col_name + '_'
    dfMovies.drop([col_name], axis=1, inplace=True)
    for col in dfMovies.columns:
        if (col.find(col_name2) == 0):
            dfMovies[col].fillna(0, inplace=True)

print("Number of columns = {}".format(len(dfMovies.columns)))

Number of columns = 248


In [9]:
# Production companies

col_key = "production_companies"
json_key = "name"
number_comp = 100
prod_map = {}
for i, row in dfMovies.iterrows():
    if (row[col_key]):
        gen = pd.read_json(row[col_key])
        if (json_key in gen.columns):
            dfMovies.loc[i, col_key] = "|".join(gen[json_key]).replace('(', '').replace(')', '')
            for v in gen[json_key]:
                if (v not in prod_map):
                    prod_map[v] = 0
                prod_map[v] += 1

print("Number of unique production companies = {}".format(len(prod_map)))
prod_map_s = sorted(prod_map.items(), key=lambda x: x[1], reverse=True)
top_n = prod_map_s[:number_comp]
print("Top {} production companies:".format(number_comp))
print(top_n)

for tup in top_n:
    name = tup[0].replace('(', '').replace(')', '')
    col = "Prod_Comp_" + name
    dfMovies[col] = 0
    dfMovies.loc[dfMovies[col_key].str.contains(name, na=False), col] = 1
dfMovies.drop([col_key], axis=1, inplace=True)

print("Number of columns = {}".format(len(dfMovies.columns)))

Number of unique production companies = 5017
Top 100 production companies:
[('Warner Bros.', 319), ('Universal Pictures', 311), ('Paramount Pictures', 285), ('Twentieth Century Fox Film Corporation', 222), ('Columbia Pictures', 201), ('New Line Cinema', 165), ('Metro-Goldwyn-Mayer (MGM)', 122), ('Touchstone Pictures', 118), ('Walt Disney Pictures', 114), ('Relativity Media', 102), ('Columbia Pictures Corporation', 96), ('Miramax Films', 94), ('Village Roadshow Pictures', 81), ('DreamWorks SKG', 79), ('Canal+', 75), ('United Artists', 75), ('Regency Enterprises', 69), ('Fox Searchlight Pictures', 64), ('Dune Entertainment', 59), ('Lionsgate', 56), ('Fox 2000 Pictures', 55), ('TriStar Pictures', 55), ('Dimension Films', 54), ('Working Title Films', 53), ('Summit Entertainment', 52), ('Amblin Entertainment', 49), ('The Weinstein Company', 42), ('StudioCanal', 41), ('New Regency Pictures', 40), ('Castle Rock Entertainment', 40), ('Screen Gems', 39), ('Spyglass Entertainment', 38), ('BBC Fi

In [10]:
# Keywords

col_key = "keywords"
json_key = "name"
number_comp = 150
prod_map = {}
for i, row in dfMovies.iterrows():
    if (row[col_key]):
        gen = pd.read_json(row[col_key])
        if (json_key in gen.columns):
            dfMovies.loc[i, col_key] = "|".join(gen[json_key]).replace('(', '').replace(')', '')
            for v in gen[json_key]:
                if (v not in prod_map):
                    prod_map[v] = 0
                prod_map[v] += 1

print("Number of unique keywords = {}".format(len(prod_map)))
prod_map_s = sorted(prod_map.items(), key=lambda x: x[1], reverse=True)
top_n = prod_map_s[:number_comp]
print("Top {} keywords:".format(number_comp))
print(top_n)

for tup in top_n:
    name = tup[0].replace('(', '').replace(')', '')
    col = "Keyword_" + name
    dfMovies[col] = 0
    rgx = "^" + name + "$|" + name + "\||\|" + name + "\||\|" + name + "$"
    dfMovies.loc[dfMovies[col_key].str.contains(name, na=False), col] = 1

dfMovies.drop([col_key], axis=1, inplace=True)

print("Number of columns = {}".format(len(dfMovies.columns)))

Number of unique keywords = 9813
Top 150 keywords:
[('woman director', 324), ('independent film', 318), ('duringcreditsstinger', 307), ('based on novel', 197), ('murder', 189), ('aftercreditsstinger', 170), ('violence', 150), ('dystopia', 139), ('sport', 126), ('revenge', 118), ('sex', 111), ('friendship', 106), ('musical', 105), ('biography', 105), ('teenager', 99), ('3d', 98), ('love', 95), ('sequel', 94), ('suspense', 92), ('new york', 89), ('police', 84), ('los angeles', 81), ('high school', 80), ('alien', 76), ('prison', 72), ('nudity', 71), ('superhero', 69), ('family', 69), ('london england', 69), ('drug', 69), ('dying and death', 66), ('father son relationship', 65), ('daughter', 61), ('world war ii', 58), ('kidnapping', 57), ('wedding', 57), ('remake', 56), ('suicide', 56), ('serial killer', 56), ('magic', 55), ('friends', 54), ('corruption', 53), ('escape', 52), ('based on comic book', 52), ('hospital', 52), ('party', 52), ('time travel', 51), ('based on true story', 51), ('m

In [11]:
dfMovies.head(30)

,budget,homepage,id,original_title,overview,popularity,release_date,revenue,runtime,status,...,Keyword_gang,Keyword_psychopath,Keyword_neighbor,Keyword_mother daughter relationship,Keyword_drug dealer,Keyword_faith,Keyword_mutant,Keyword_hero,Keyword_rivalry,Keyword_california
0,237000000,http://www.avatarmovie.com/,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2787965087,162.0,Released,...,0,0,0,0,0,0,0,0,0,0
1,300000000,http://disney.go.com/disneypictures/pirates/,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,961000000,169.0,Released,...,0,0,0,0,0,0,0,0,0,0
2,245000000,http://www.sonypictures.com/movies/spectre/,206647,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,880674609,148.0,Released,...,0,0,0,0,0,0,0,0,0,0
3,250000000,http://www.thedarkknightrises.com/,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,1084939099,165.0,Released,...,0,0,0,0,0,0,0,1,0,0
4,260000000,http://movies.disney.com/john-carter,49529,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,284139100,132.0,Released,...,0,0,0,0,0,0,0,0,0,0
5,258000000,http://www.sonypictures.com/movies/spider-man3/,559,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,115.699814,2007-05-01,890871626,139.0,Released,...,0,0,0,0,0,0,0,1,0,0
6,260000000,http://disney.go.com/disneypictures/tangled/,38757,Tangled,When the kingdom's most wanted-and most charmi...,48.681969,2010-11-24,591794936,100.0,Released,...,0,0,0,0,0,0,0,0,0,0
7,280000000,http://marvel.com/movies/movie/193/avengers_ag...,99861,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,2015-04-22,1405403694,141.0,Released,...,0,0,0,0,0,0,0,1,0,0
8,250000000,http://harrypotter.warnerbros.com/harrypottera...,767,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...",98.885637,2009-07-07,933959197,153.0,Released,...,0,0,0,0,0,0,0,0,0,0
9,250000000,http://www.batmanvsupermandawnofjustice.com/,209112,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,155.790452,2016-03-23,873260194,151.0,Released,...,0,0,0,0,0,0,0,1,0,0


In [12]:
dfMovies.to_csv('tmdb_5000_movies_out_final.csv', sep=',', header=True)